# Prepare all lists for later use
- Make a list with all the solvents and their hazardenous
- Make a list of all GHS/H/P codes
- Compile a list of H_code to severity

## Libraries

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

## Functions and definitions

In [3]:
# No standard functions needed

## Main part

### Solvent file
Make a easy readable list showing the toxicity for each solvent

In [4]:
#read all solvents and their toxicities according to Bryne et al.
#this list was compiled using the previously created list in 1_Create_List_All_Solvents.ipynb
solvents = pd.read_excel('../data/raw/List_of_all_solvents_MCcompiled_MMUannot_Aug2019.xlsx')

In [5]:
# remove last row (just comment)
solvents = solvents.drop(25)

In [9]:
#read all solvents
all_solvents = (solvents.values[:,0])
#read all toxicities (values between 1 and 6)
all_values = (solvents.values[:,1])

In [5]:
#This dictionary contains the toxicity for each solvent (according to Miriam)
solvent_to_value = {}

#go through all solvent names and toxicites and assign them correctly (for mixtures for now assign individual values)
for solv, val in zip(all_solvents,all_values):
    solvs = solv.split(';')

    vals = str(val).split(';')
    for s,v in zip(solvs,vals):

        solvent_to_value[s] = int(v)
print (solvent_to_value)

{'methanol': 1, ' acetic acid': 2, 'chlorobenzene': 3, '1,4-dioxane': 5, ' N,N-dimethyl-formamide': 5, ' water': 1, 'nitromethane': 6, 'water': 1, ' glycerol': 1, ' toluene': 3, ' acetonitrile': 3, 'ethyl acetate': 2, 'acetonitrile': 3, 'dichloromethane': 4, 'dimethyl sulfoxide': 3, 'ethanol': 1, 'chloroform': 6, 'glycerol': 1, '1,2-dichloro-ethane': 6, 'acetic acid': 2, 'toluene': 3, 'neat (no solvent)': 1, ' chloroform': 6}


In [6]:
#Make output list
fp_out = open('../data/processed/List_of_Solvents.tsv','w')
fp_out.write('Solvent\thazardousness\n')
for key in solvent_to_value:
    fp_out.write(key+'\t'+str(solvent_to_value[key])+'\n')
fp_out.close()

## Make list of H_code to toxicity

Hazard codes have either categories (mostly), divisions or other 
see https://www.era-environmental.com/blog/ghs-hazard-classification

- Categories easiest with 1 being the most severe, 4 the least
- Division similar easy
- Other such as "Gases under pressure" got a middle severity since the Signal word is not danger typiacally corresponding to category 3 and 4
- Quite rare category 5 exists (special case --> weigthed 0.1, i.e., lower than 1/4

In [7]:
category_to_severity = {
    
    #Physical hazzards
    'Explosives' : {'Unstable Explosive':6/6, 'Div 1.1':6/6, 'Div 1.2':5/6, 'Div 1.3':4/6, 'Div 1.4':3/6, 'Div 1.5':2./6, 'Div 1.6':1./6},
    'Desensitized explosives' : {'Category 1':4/4, 'Category 2':3/4,'Category 3':2/4, 'Category 4':1/4},
    'Flammable gases': {'1A: Flammable gas, Pyrophoric gas, Chemically unstable gas A,B':1.0, '1B':1.0,'1A, Pyrophoric gas':1.0, '1A, Chemically unstable gas A':0.5, '1A, Chemically unstable gas B':0.5, 'Category 2':3/4},
    'Aerosols': {'Category 1':4/4, 'Category 2':3/4, 'Category 3':2/4},
    'Flammable liquids': {'Category 1':4/4, 'Category 2':3/4,'Category 3':2/4, 'Category 4':1/4},
    'Flammable solids' : {'Category 1':4/4, 'Category 2':3/4},
    'Self-reactive substances and mixtures; Organic peroxides' : {'Type A':5/5, 'Type B':4/5,'Type C, D':3/5, 'Type E, F':2/5, 'Type G':1/5},
    'Pyrophoric liquids; Pyrophoric solids' : {'Category 1':4/4},
    'Self-heating substances and mixtures' : {'Category 1':4/4, 'Category 2':3/4},
    'Substances and mixtures which in contact with water, emit flammable gases': {'Category 1':4/4, 'Category 2':3/4,'Category 3':2/4},
    'Oxidizing gases': {'Category 1':4/4},
    'Oxidizing liquids; Oxidizing solids' : {'Category 1':4/4, 'Category 2':3/4,'Category 3':2/4},
    'Gases under pressure': {'Compressed gas, Liquefied gas, Dissolved gas':0.5, 'Refrigerated liquefied gas':0.5 },
    'Chemicals under pressure' : {'Category 1':4/4, 'Category 2':3/4, 'Category 3':2/4},
    'Corrosive to Metals' : {'Category 1':4/4},
    
    
    #Health hazzards
    'Acute toxicity, oral' : {'Category 1, 2':4/4, 'Category 3':2/4,'Category 4':1/4, 'Category 5':0.1}, ##Contains Category 5
    'Aspiration hazard' : {'Category 1':4/4, 'Category 2':3/4},
    'Acute toxicity, dermal' : {'Category 1, 2':4/4, 'Category 3':2/4,'Category 4':1/4, 'Category 5':0.1}, #Contains Category 5
    'Skin corrosion/irritation' : {'Category 1A, 1B, 1C':4/4, 'Category 2':3/4, 'Category 3':2/4},
    'Sensitization, Skin' : {'Category 1, 1A, 1B' :4/4},
    'Serious eye damage/eye irritation' : {'Category 1':4/4, 'Category 2A':3/4, 'Category 2B':2/4},
    'Acute toxicity, inhalation': {'Category 1, 2':4/4, 'Category 3':2/4,'Category 4':1/4, 'Category 5':0.1}, #Contains Category 5
    'Sensitization, respiratory': {'Category 1, 1A, 1B':4/4},
    'Specific target organ toxicity, single exposure; Respiratory tract irritation' : {'Category 3':2/4},
    'Specific target organ toxicity, single exposure; Narcotic effects': {'Category 3':2/4},
    'Germ cell mutagenicity': {'Category 1A, 1B':4/4, 'Category 2':3/4},
    'Carcinogenicity': {'Category 1A, 1B':4/4, 'Category 2':3/4},
    'Reproductive toxicity': {'Category 1A, 1B':4/4, 'Category 2':3/4},
    'Reproductive toxicity, effects on or via lactation' : { 'Additional category': 1/4},
    'Specific target organ toxicity, single exposure': {'Category 1':4/4, 'Category 2':3/4},
    'Specific target organ toxicity, repeated exposure' : {'Category 1':4/4, 'Category 2':3/4},
    
    #Environment hazzards
    'Hazardous to the aquatic environment, acute hazard' : {'Category 1':4/4, 'Category 2':3/4, 'Category 3':2/4},
    'Hazardous to the aquatic environment, long-term hazard' : {'Category 1':4/4, 'Category 2':3/4, 'Category 3':2/4, 'Category 4':1/4},
    'Hazardous to the ozone layer': {'Category 1':4/4},


    #Combined H_codes
    'H300+H310': 4/4,       #Category 1
    'H300+H330': 4/4,       #Category 1
    'H310+H330': 4/4,       #Category 1
    'H300+H310+H330': 4/4,  #Category 1
    
    'H301+H311': 2/4,       #Category 3
    'H301+H331': 2/4,       #Category 3
    'H311+H331': 2/4,       #Category 3
    'H301+H311+H331': 2/4,  #Category 3
    
    'H302+H312': 1/4,       #Category 4
    'H302+H332': 1/4,       #Category 4
    'H312+H332': 1/4,       #Category 4
    'H302+H312+H332': 1/4,  #Category 4
    
    'H303+H313': 0.1,        #Category 5
    'H303+H333': 0.1,        #Category 5
    'H313+H333': 0.1,        #Category 5
    'H303+H313+H333': 0.1,   #Category 5
    
    'H315+H320': 3/4         #Category 2, 2B	
    
}
    

In [8]:
#read the .xslx file with the corresponding categories/divisions etc.
H_code_list = pd.read_excel('../data/raw/Hcodes_withCategories.xlsx')
H_code_list.head()

,Code,Hazard Statements,Hazard Class,Category,Signal Word,Prevention,Response,Storage,Disposal
0,H200,Unstable Explosive,Explosives,Unstable Explosive,Danger,"P201, P202, P281","P372, P373, P380",P401,P501
1,H201,Explosive; mass explosion hazard,Explosives,Div 1.1,Danger,"P210, P230, P240, P250, P280","P370+P380, P372, P373",P401,P501
2,H202,Explosive; severe projection hazard,Explosives,Div 1.2,Danger,"P210, P230, P240, P250, P280","P370+P380, P372, P373",P401,P501
3,H203,"Explosive; fire, blast or projection hazard",Explosives,Div 1.3,Danger,"P210, P230, P240, P250, P280","P370+P380, P372, P373",P401,P501
4,H204,Fire or projection hazard,Explosives,Div 1.4,Warning,"P210, P240, P250, P280","P370+P380, P372, P373, P374",P401,P501


In [9]:
#final dictionary that contains the H-code severity
H_code_to_severity = {}

#go through all H-codes
for row in H_code_list.iterrows():
    
    #get hcode
    hcode = row[1][0]
    
    #if 'empty' row (e.g. group header for a class of hazard codes)
    if str(hcode) == 'nan':
        continue

    #get other values
    hazard_class = row[1][2]
    category = row[1][3]
    signal_word = row[1][4]

    #some hcodes are joints of several hcodes; those joint H-codes got exact severity classes
    if '+' not in hcode:
        H_code_to_severity[hcode] = category_to_severity[hazard_class][category]
    else:
        H_code_to_severity[hcode] = category_to_severity[hcode]
    

In [10]:
#create a file with a severity for each H-code
fp_out = open('../data/processed/HCode_Severity.csv','w')
fp_out.write('Hcode,Severity\n')
for code in H_code_to_severity:
    fp_out.write(code+','+str(H_code_to_severity[code])+'\n')
fp_out.close()

### Catalyst File

In [11]:
#open the manual annotated catalyst file from Miriam
catalyts = pd.read_excel('../data/raw/Catalysts.xlsx')

#drop the first two columns (just comments)
catalyts = catalyts.drop(0)
catalyts = catalyts.drop(1)

catalyts = catalyts.rename(columns=catalyts.iloc[0])
catalyts = catalyts.drop(2)
catalyts.tail(5)

,who annotated,1 compound or mixture/hybrid,REAGENTS,individual components if multicomponent reagent,Hazard [GHS = Globally Harmonized System of Classification and Labelling of Chemicals],H-sentences [Hazard statements],P-sentences [precautionary statements],comment,NaN,NaN,NaN
185,Fabián,1,1-ethyl-3-methylimidazolium triflate,NaN,GHS07,"H315, H319","P264, P280, P305+P351+P338, P321+P332+P313, P3...",https://en.solvionic.com/files/solvionic/fds/A...,NaN,NaN,NaN
186,Miriam,mixture,ZnO-loaded mesoporous silica (KIT-6) (aged at ...,ZnO,X,X,X,KIT-6 saefty is ffrom here: https://www.acsmat...,NaN,NaN,NaN
187,NaN,NaN,NaN,SiO2,X,X,X,NaN,NaN,NaN,NaN
188,NaN,NaN,NaN,KIT-6,GHS08,H372,"P26, P264, P270, P280, P314, P403+P233, P405, ...",NaN,NaN,NaN,NaN
189,Fabián,1,magnesium sulfate heptahydrate,MgSO4 7H2O,X,X,X,PubChem: The chemical has been verified to be ...,NaN,NaN,NaN


Make Miriam excel sheet into easy readable .tsv file

In [12]:
#some catalysts are mixtures of chemicals (then typically the first entry then entries have 'nan' and belong to previous entry)
catalyst_id = 0
current_cat = None

catalysators = {}

for row in catalyts.iterrows():
    
    #Get name, if it is a composite catalyst and the component name
    current_cat = row[1][2]
    is_mix = row[1][1]
    comp_name = row[1][3]
    
    #Get comment
    comment = row[1][7]
    if str(comment) == 'nan':
        comment = 'None'
    comment = comment.strip()


    #Get source
    source = row[1][8]
    if str(source) == 'nan':
        source = 'None'
    source = source.strip()
    
    
    #Get GSH codes
    GSH_codes = [x.strip() for x in row[1][4].split(',') if x != 'X' and x != 'x']
    
    #Get H_Codes (flatten codes combined with Plus)
    H_codes =[x.strip() for x in row[1][5].split(',') if x != 'X' and x != 'x']  
    
    #flatten the whole list
    H_flatten = []
    for H in H_codes:
        H_flatten.append(H.replace(' ',''))
        
            
    
    #Get H_Codes (flatten codes combined with Plus)
    P_codes =[x.strip() for x in row[1][6].split(',') if x != 'X' and x != 'x']  
    #P_flatten = P_codes
    
    P_flatten = []
    for P in P_codes:
        P_flatten.append(P.replace(' ',''))

    

    #if to_annotate == 'nan' it means it is part of a mixture (mixture entries have the first row annotated)
    if str(row[1][0]) != 'nan':
        #if is has a name it is either the start of a mixture (or a single agent catalyst)
        
        #set catalyst and component_ID
        catalyst_id +=1
        compononent_ID = 0
        
        if is_mix == 'mixture':
            catalysators[catalyst_id]= {'Catalyst_Name':current_cat,'Comment':comment,'Is_Mixture':'YES', compononent_ID:{'Component_Name':comp_name, 'GHS':GSH_codes, 'H':H_flatten, 'P':P_flatten, 'Source':source}}
        else:
            catalysators[catalyst_id]= {'Catalyst_Name':current_cat,'Comment':comment,'Is_Mixture':'NO', compononent_ID:{'Component_Name':current_cat, 'GHS':GSH_codes, 'H':H_flatten, 'P':P_flatten, 'Source':source}}
    
    #if here it means further entries of the same catalyst
    else:
        compononent_ID += 1
        catalysators[catalyst_id][compononent_ID] = {'Component_Name':comp_name, 'GHS':GSH_codes, 'H':H_flatten, 'P':P_flatten, 'Source':source}

In [14]:
# CHECK Original List with Miriam/Fabian List (of names of catalysts)
list_original_cats = []

#open list previously compiled in 1_Create_List_All_Solvents.ipynb (containing all reagents)
fp = open('../results/1_Solvent_Catalyst_Lists/List_of_all_reagents_and_catalysts.tsv')
for line in fp:
    cat = line.strip()
    list_original_cats.append(cat)
    
    found = False
    for i in range(1, max(catalysators.keys())+1):
        if cat == catalysators[i]['Catalyst_Name']:
            found = True
            
    if found == False:
        print (cat)
        
# IF not a single NOT FOUND then perfect! This means all catalyts/reagents have been correctly manual annotated.

In [15]:
for i in range(1, max(catalysators.keys())+1):
    if catalysators[i]['Catalyst_Name'] not in list_original_cats:
        print (catalysators[i]['Catalyst_Name'])
# USE to adjust names

#### Make final output file

In [16]:
# MAKE OUTPUT FILE
fp_out = open('../data/processed/Catalysts_Overview.tsv','w')
fp_out.write('Catalyst_ID\tCatalyst_Name\tIs_Mixture\tComponent_ID\tComponent_Name\tGSH_Codes\tH_Codes\tP_Codes\tSource\tComments\n')

#Go through all catalystators
for i in range(1, max(catalysators.keys())+1):
    
    #go through all components
    i2= 0
    while i2 in catalysators[i]:
        
        #print (catalysators[i]['Catalyst_Name'])
        
        fp_out.write(str(i)+'\t'+catalysators[i]['Catalyst_Name']+'\t'+catalysators[i]['Is_Mixture'] +'\t'
                    + str(i2) + '\t' + catalysators[i][i2]['Component_Name'] + '\t' + ';'.join(catalysators[i][i2]['GHS'])
                    + '\t' + ';'.join(catalysators[i][i2]['H']) + '\t' + ';'.join(catalysators[i][i2]['P'])
                    + '\t' + catalysators[i][i2]['Source'] + '\t' + catalysators[i]['Comment'] + '\n')
        
        i2 +=1
    
fp_out.close()

### Read condition file

In [17]:
interesting_conditions = []
fp = open('../results/1_Solvent_Catalyst_Lists/List_of_interesting_conditions.tsv')
for line in fp:
    interesting_conditions.append(line.strip())
print (interesting_conditions)

['Reflux', 'Green chemistry', 'Neat (no solvent)', 'Sonication', 'neat (no solvent)', 'Heating', 'Irradiation', 'Microwave irradiation']


### Create a complete list of all reactions used in this study

In [28]:
#selected reaction conditions
important_columns = ['Temperature (Reaction Details) [C]','Time (Reaction Details) [h]','Solvent (Reaction Details)','Catalyst','Reagent','Yield (numerical)','Other Conditions','Number of Reaction Steps','Reaction: Links to Reaxys','References']

#open empty dataframe with seleted parameters, i.e., important_columns
all_compounds = pd.DataFrame(columns=important_columns)
all_compounds['Compound_ID'] = ''
all_compounds['Type'] = ''

#go through all compounds, add all found reactions to the initialized empty dataframe
mypath = '../data/raw/compounds/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and '~' not in f]
for file in onlyfiles:

    print (file)

    #red the excel file
    compound = pd.read_excel(mypath+file)

    #differentiate between literature and real(Fabians) conditions
    fabian_condition = compound.iloc[len(compound)-3,][important_columns]
    compound = compound.iloc[0:len(compound)-3,][important_columns]
    
    
    #get current compound id
    compound_id = file.split('.')[0].split('_')[2]

    
    #add special row to distinguish the example and fabian conditions
    compound['Type'] = 'Other'
    fabian_condition['Type'] = 'Fabian'
    compound = compound.append(fabian_condition)

    #make Tmp and time to string
    compound['Temperature (Reaction Details) [C]'] = compound['Temperature (Reaction Details) [C]'].astype(str)
    compound['Time (Reaction Details) [h]'] = compound['Time (Reaction Details) [h]'].astype(str)
    
    
    #take higher value for ranges
    compound.loc[compound['Temperature (Reaction Details) [C]'].str.contains(';', na=False), 'Temperature (Reaction Details) [C]'] = compound.loc[compound['Temperature (Reaction Details) [C]'].str.contains(';', na=False)]['Temperature (Reaction Details) [C]'].str.split(';').str[1]
    compound.loc[compound['Temperature (Reaction Details) [C]'].str.contains(' - ', na=False), 'Temperature (Reaction Details) [C]'] = compound.loc[compound['Temperature (Reaction Details) [C]'].str.contains(' - ', na=False)]['Temperature (Reaction Details) [C]'].str.split(' - ').str[1]
    compound.loc[compound['Time (Reaction Details) [h]'].str.contains(';', na=False), 'Time (Reaction Details) [h]'] = compound.loc[compound['Time (Reaction Details) [h]'].str.contains(';', na=False)]['Time (Reaction Details) [h]'].str.split(';').str[1]
    
    #set 'catalyst' to true/false
    
    #compound.Catalyst.loc[(compound['Catalyst'] != False)] = 1
    compound = compound.fillna(value = {'Catalyst':''})
    compound = compound.fillna(value = {'Reagent':''})
    
    
    #replace empty/nan values with the unique 'None' as identifier for missing values
    compound = compound.fillna('None')
    compound = compound.replace({'nan': 'None'})

    
    

    compound['Reagent/Catalyst'] = compound['Catalyst'] + compound['Reagent']
    compound.loc[compound['Reagent'] == '', 'Reagent'] = 'None'
    compound.loc[compound['Catalyst'] == '', 'Catalyst'] = 'None'
    compound.loc[compound['Reagent/Catalyst'] == '', 'Reagent/Catalyst'] = 'None'
    #compound = compound.fillna(value = {'Reagent':''})
    
    
    #add compound ID
    compound['Compound_ID'] = compound_id

    for x in compound.iterrows():
        
        conditions =  x[1][6].split(';')

        for c in conditions:
            c = c.strip()
            if c not in interesting_conditions:
                #print (c)
                compound.iloc[x[0]]['Other Conditions'] = 'None'

    
    #add current compound to all compounds
    all_compounds = all_compounds.append(compound)
    
#save final csv file
all_compounds.to_csv('../data/processed/All_Reactions.csv')
compound.head()  

Reaxys_Compound_6.xlsx


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Reaxys_Compound_7.xlsx
Reaxys_Compound_10.xlsx
Reaxys_Compound_1.xlsx
Reaxys_Compound_11.xlsx
Reaxys_Compound_2.xlsx
Reaxys_Compound_12.xlsx
Reaxys_Compound_13.xlsx
Reaxys_Compound_3.xlsx
Reaxys_Compound_4.xlsx
Reaxys_Compound_14.xlsx
Reaxys_Compound_8.xlsx
Reaxys_Compound_9.xlsx
Reaxys_Compound_5.xlsx


,Temperature (Reaction Details) [C],Time (Reaction Details) [h],Solvent (Reaction Details),Catalyst,Reagent,Yield (numerical),Other Conditions,Number of Reaction Steps,Reaction: Links to Reaxys,References,Type,Reagent/Catalyst,Compound_ID
0,20,0.166667,ethanol,None,gallium(III) triflate,100,None,1.0,https://www.reaxys.com/reaxys/secured/hopinto....,"Article; Cai, Jing-Jing; Zou, Jian-Ping; Pan, ...",Other,gallium(III) triflate,5
1,20,0.00416667,ethyl acetate,None,silica supported 12-tungstophosphoric acid nan...,99,None,1.0,https://www.reaxys.com/reaxys/secured/hopinto....,"Article; Abdollahi-Alibeik, Mohammad; Heidari-...",Other,silica supported 12-tungstophosphoric acid nan...,5
2,120,0.666667,None,None,None,99,Green chemistry,1.0,https://www.reaxys.com/reaxys/secured/hopinto....,"Article; Huang, Tie-Qiang; Qu, Wen-Yan; Ding, ...",Other,None,5
3,80,0.25,neat (no solvent),None,Cs(cetyltrimethylammonium)2PW12O40,99,Green chemistry,1.0,https://www.reaxys.com/reaxys/secured/hopinto....,"Article; Masteri-Farahani, Majid; Ghorbani, Me...",Other,Cs(cetyltrimethylammonium)2PW12O40,5
4,120,0.05,None,None,None,98,Microwave irradiation,1.0,https://www.reaxys.com/reaxys/secured/hopinto....,"Article; Zhou, Jian-Feng; Gong, Gui-Xia; Zhi, ...",Other,None,5
